In [352]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
import datetime as datetime
import pandas as pd
import plotly.express as px
import urllib.request
pn.extension('plotly')


In [353]:

###Data sourced from https://data.covid19india.org/
### District Wise Data
districts_url = "https://data.covid19india.org/csv/latest/districts.csv"
districts_filename = "districts.csv"

#Download if not present
#urllib.request.urlretrieve(districts_url, districts_filename)

data = pd.read_csv(districts_filename)

### Vaccinatin Data
vaccine_url = "http://data.covid19india.org/csv/latest/vaccine_doses_statewise_v2.csv"
vaccine_filename = "vaccine_doses_statewise_v2.csv"

#Download if not present
#urllib.request.urlretrieve(vaccine_url, vaccine_filename)

vacdata = pd.read_csv(vaccine_filename)


In [354]:
#state-date wise data
data['Active'] = data['Confirmed'] - data['Recovered'] - data['Deceased']
state_wise_cumulative = data.groupby(['State','Date'])[['Confirmed','Recovered','Deceased','Active']].sum()
state_wise_delta = state_wise_cumulative.diff()
state_wise_delta[state_wise_delta<0] = None

#vaccine data
vacdata.rename(columns = {'Vaccinated As of':'Date'}, inplace = True)
vacdata['Date'] = pd.to_datetime(vacdata['Date'])
vaccine_state_wise = vacdata.groupby(['State','Date'])['Total Doses Administered'].sum()
vaccine_data_delta = vaccine_state_wise.diff()

#plotting data
plot_data = state_wise_delta
plot_data['Active'] = state_wise_cumulative['Active']
plot_data.rename(columns = {'Confirmed':'New Cases'}, inplace = True)



C:\Users\umesh\AppData\Local\Temp\ipykernel_8640\4134050494.py:9: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [355]:
def vaccine_data(date_str, state, ans):
    ans['Vaccinated'] = 0
    try:
        for st in state:
            vaccinated = vaccine_data_delta.loc[(st, date_str)].astype(int)
            if vaccinated >= 0:
                ans['Vaccinated'] += vaccinated
    except:
        ans['Vaccinated'] = "NA"

def daily_data(date_str, state, ans):
    ans['New Cases'] = 0
    ans['Recovered'] = 0
    ans['Deceased'] = 0
    try:
        for st in state:
            new_data = state_wise_delta.loc[(st, date_str)].astype(int)
            ans['New Cases'] += new_data['New Cases'].astype(int)
            ans['Recovered'] += new_data['Recovered']
            ans['Deceased'] += new_data['Deceased']
    except:
        ans['New Cases'] = "NA"
        ans['Recovered'] = "NA"
        ans['Deceased'] = "NA"
def active_cases(date_str, state, ans):
    active_cases = 0
    try:
        for st in state:
            res = state_wise_cumulative.loc[(st, date_str)]
            active_cases = active_cases + res.iloc[0] - res.iloc[1] - res.iloc[2]

    except:
        active_cases = "NA"
    ans['Active Cases'] = active_cases

    
#top 10 districts 
def gettop10(date, state):
    date_str = date.strftime("%Y-%m-%d")
    try:
        ans = data.loc[data['State'].isin(state)][data['Date'] == date_str].sort_values('Active',ascending = False).head(10)[['District', 'Active']]
        ans = ans.set_index('District')
        ans.rename(columns = {'Active' : 'Most Active Cases by Districts(Top 10)'}, inplace = True)
    except:
        ans = "Cannot find"
    return ans
        
def data_retriever(date, state):
    date_str = date.strftime("%Y-%m-%d")

    ans = pd.Series(name = "Data for : " +  str(date_str))
    
    active_cases(date_str, state, ans)
    daily_data(date_str, state, ans)
    vaccine_data(date_str, state, ans)
    #gettop10(date_str, state, ans)
    
    return ans

def getChart(state, metric):
    chart_data = plot_data.loc[state].reset_index()
    chart_data = chart_data.loc[chart_data['Date'] > '2020-12-31'].loc[chart_data['Date'] < '2021-12-31']
    fig = px.line()
    for st in state:
        st_data = chart_data.loc[chart_data['State'] == st]
        fig.add_scatter(name=st,x=st_data['Date'], y=st_data[metric])
    fig.update_traces(mode="lines+markers", marker=dict(size=2), line=dict(width=1))
    fig.layout.autosize = True
    responsive = pn.pane.Plotly(fig)
    return responsive
    


In [356]:
#selectors
date_picker = pn.widgets.DatePicker(name='Pick Date', value = datetime.date(2021, 1, 29), start = datetime.date(2021,1,1), end = datetime.date(2021,12,31))
#state_picker = pn.widgets.Select(name="State", value="Maharashtra", options=list(data["State"].unique()))
state_picker = pn.widgets.MultiChoice(name='Select State', value=['Maharashtra', 'Bihar'], options=list(data["State"].unique()))
#day-state wise data widgets
data_display = pn.bind(data_retriever, date=date_picker, state = state_picker)
top_districts = pn.bind(gettop10, date=date_picker, state = state_picker)

#Day-State wise data widget placing
col1 = pn.Column(date_picker, state_picker, width = 400)
display1 = pn.Row(col1, data_display, top_districts)



# plotting charts
active_chart = pn.bind(getChart, state = state_picker, metric = 'Active')
new_cases_chart = pn.bind(getChart, state = state_picker, metric = 'New Cases')
recovered_chart = pn.bind(getChart, state = state_picker, metric = 'Recovered')
deceased_chart = pn.bind(getChart, state = state_picker, metric = 'Deceased')

# Grouping Active Cases and New Cases
chart_row1 = pn.Row(active_chart, new_cases_chart, height = 400)

# Grouping Recovered and Deceased
chart_row2 = pn.Row(recovered_chart, deceased_chart, height = 400)


first_app = pn.Column(display1, chart_row1, chart_row2)
first_app.servable()


C:\Users\umesh\AppData\Local\Temp\ipykernel_8640\756253062.py:41: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Column
    [0] Row
        [0] Column(width=400)
            [0] DatePicker(end=datetime.date(2021, ..., name='Pick Date', start=datetime.date(2021, 1, 1), value=datetime.date(2021, ...)
            [1] MultiChoice(name='Select State', options=['Andaman and Nicobar Isla...], value=['Maharashtra', 'Bihar'])
        [1] ParamFunction(function, _pane=DataFrame, defer_load=False)
        [2] ParamFunction(function, _pane=DataFrame, defer_load=False)
    [1] Row(height=400)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [1] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] Row(height=400)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)
        [1] ParamFunction(function, _pane=Plotly, defer_load=False)